<a href="https://colab.research.google.com/github/MrAFirm/Introduction-to-Data_Analytics_-_ML/blob/main/Face_Recogniser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# Face Training for Facial Recognition

import os
import cv2
import numpy as np
from google.colab import files
import zipfile

people = ["Shaun", "Kobe"]
# uploaded = files.upload()

DIR = "train_data1/train_data/Train"

# Haar Cascade for Facial Detection.
haar_cascade = cv2.CascadeClassifier('haar_cascade.xml')

with zipfile.ZipFile('train_data1.zip', 'r') as zip_ref:
  zip_ref.extractall("train_data1")

# for root, dirs, files in os.walk("train_data"):
#     for name in dirs:
#         print("📁", os.path.join(root, name))
#     for name in files:
#         print("📄", os.path.join(root, name))

features = []
labels = []

# Allowing LBPH face recogniser to train on the faces by detection.
def create_train():
  for person in people:
    path = os.path.join(DIR, person)
    label = people.index(person)

    for img in os.listdir(path):
      img_path = os.path.join(path, img)
      img_array = cv2.imread(img_path)
      gray = cv2.cvtColor(img_array , cv2.COLOR_BGR2GRAY)
      faces_rect = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)

      for (x, y, w, h) in faces_rect:
        faces_roi = gray[y:y+h, x:x+w]
        resized_face = cv2.resize(faces_roi, (100, 100))
        features.append(resized_face)
        labels.append(label)

create_train()
print("Training Done!")

# 45 faces in total for training data.
print(f"Length of the features = {len(features)}")
print(f"Length of the labels = {len(labels)}")

# To check if the resolution of each image is correct
print(type(features[0]))
print(features[0].shape)

features = np.array(features, dtype = np.uint8)
labels = np.array(labels)

# Call the training function
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.train(features, labels)


# Save data
face_recognizer.save("face_trained.yml")
np.save('features.npy', features)
np.save('labels.npy', labels)




Training Done!
Length of the features = 51
Length of the labels = 51
<class 'numpy.ndarray'>
(100, 100)


In [ ]:
# To install OpenCV LBPH built-in facial Recognition Library

!pip install opencv-contrib-python --upgrade


In [57]:
# Facial Recognition Program

import cv2
import numpy as np
from google.colab import files

# uploaded = files.upload()

img_name_n_path = "train_data1/train_data/Recog/Kobe/kobe-1.jpg"

# Haar Cascade for Facial Detection.
haar_cascade = cv2.CascadeClassifier('haar_cascade.xml')

people = ["Shaun", "Kobe"]
features = np.load('features.npy')
labels = np.load('labels.npy')

face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.read("face_trained.yml")

img = cv2.imread(img_name_n_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect Face
faces_rect = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=6)

for (x, y, w, h) in faces_rect:
  faces_roi = gray[y:y+h, x:x+w]
  label, confidence = face_recognizer.predict(faces_roi)
  print(f"Label = {people[label]} with a confidence of {confidence}")

  # cv2.putText(img, str(people[label]), (20,20), cv2.FONT_HERSHEY_COMPLEX, 1.0, (0,255,0), thickness = 2)
  # cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), thickness = 2)

# cv2.imshow("Detected Face", img)
# cv2.waitKey(0)

Label = Kobe with a confidence of 108.45006218408997
